# OpenAI Agent - Reporting Weather Forecast

## An Agent that Reads Weather

In [94]:
import os
import uuid
import requests
from dotenv import load_dotenv
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Optional

from agents import function_tool, RunContextWrapper
from agents import Agent, Runner

# ---------------------------------------------------------
# Load environment variables (.env must contain OPENWEATHER_API_KEY)
# ---------------------------------------------------------
load_dotenv()


# ---------------------------------------------------------
# Dataclasses for structured weather output
# ---------------------------------------------------------

@dataclass
class DailyWeather:
    """Represents a single day's weather report."""
    date: str
    condition: str
    temperature: float
    humidity: int
    wind: float


@dataclass
class WeatherForecast:
    """Represents a multi-day weather forecast for a city."""
    city: str
    daily: List[DailyWeather] = field(default_factory=list)


@dataclass
class MessageState:
    """
    State object provided to the agent.
    Stores the session ID and any weather forecast returned by the tool.
    """
    session_id: str
    weather_forecast: Optional[WeatherForecast] = None


# ---------------------------------------------------------
# Tool: Fetch weather forecast from OpenWeatherMap
# ---------------------------------------------------------

@function_tool
def get_weather_forecast(state: MessageState, city: str, date: Optional[str] = None) -> str:
    """
    Fetch a 5-day weather forecast for a city using OpenWeatherMap.
    If 'date' (YYYY-MM-DD) is provided, the result is filtered to that date only.
    """
    print(f"[DEBUG] get_weather_forecast called with city={city}, date={date}")

    api_key = os.getenv("OPENWEATHER_API_KEY")
    if not api_key:
        return "Error: OPENWEATHER_API_KEY not found in environment."

    url = "https://api.openweathermap.org/data/2.5/forecast"

    try:
        response = requests.get(
            url,
            params={"q": city, "appid": api_key, "units": "metric"},
            timeout=10
        )
        data = response.json()
        print(f"[DEBUG] Raw OpenWeatherMap response: {data}")
    except Exception as e:
        return f"Error calling weather API: {str(e)}"

    # API failed
    if data.get("cod") != "200":
        return f"Error fetching weather: {data.get('message', 'Unknown error')}"

    daily_forecast_dict = {}

    # Process the 3-hour interval forecast entries
    for entry in data["list"]:
        dt_txt = entry["dt_txt"].split(" ")[0]  # Extract date YYYY-MM-DD

        # If user requested a specific date, filter out all others
        if date and dt_txt != date:
            continue

        if dt_txt not in daily_forecast_dict:
            daily_forecast_dict[dt_txt] = DailyWeather(
                date=dt_txt,
                condition=entry["weather"][0]["description"].capitalize(),
                temperature=entry["main"]["temp"],
                humidity=entry["main"]["humidity"],
                wind=entry["wind"]["speed"]
            )

    forecast = WeatherForecast(city=city, daily=list(daily_forecast_dict.values()))
    state.weather_forecast = forecast

    # If user requested a specific date but none existed
    if date and not forecast.daily:
        return f"No forecast available for {city} on {date}. (API supports only the next 5 days)"

    # Build a human-readable summary
    summary_lines = [f"Weather forecast for {city}:"]
    for day in forecast.daily:
        summary_lines.append(
            f"{day.date}: {day.condition}, Temp: {day.temperature}°C, "
            f"Humidity: {day.humidity}%, Wind: {day.wind} m/s"
        )

    return "\n".join(summary_lines)


# ---------------------------------------------------------
# Agent Setup
# ---------------------------------------------------------

weather_agent = Agent[MessageState](
    name="Weather Forecast Agent",
    instructions="""
    You are an assistant that provides weather forecasts.
    Use the get_weather_forecast tool to retrieve weather information.
    """,
    model="gpt-4o-mini",
    tools=[get_weather_forecast]
)

# ---------------------------------------------------------
# Run the agent with test input
# ---------------------------------------------------------

context = MessageState(session_id=str(uuid.uuid4()))

response = await Runner.run(
    weather_agent,
    [{"role": "user", "content": "Find me the weather forecast for Dallas on 2025-11-25."}],
    context=context
)

# Show agent output
print("[DEBUG] Agent final output:", response.final_output)

# Show structured forecast object
if context.weather_forecast:
    for day in context.weather_forecast.daily:
        print(
            f"{day.date}: {day.condition}, Temp: {day.temperature}°C, "
            f"Humidity: {day.humidity}%, Wind: {day.wind} m/s"
        )


[DEBUG] get_weather_forecast called with city=Dallas, date=2025-11-25
[DEBUG] Raw OpenWeatherMap response: {'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1763866800, 'main': {'temp': 14.12, 'feels_like': 13.42, 'temp_min': 12.81, 'temp_max': 14.12, 'pressure': 1021, 'sea_level': 1021, 'grnd_level': 1004, 'humidity': 70, 'temp_kf': 1.31}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02n'}], 'clouds': {'all': 22}, 'wind': {'speed': 1.72, 'deg': 32, 'gust': 4.31}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-11-23 03:00:00'}, {'dt': 1763877600, 'main': {'temp': 13.16, 'feels_like': 12.23, 'temp_min': 12.35, 'temp_max': 13.16, 'pressure': 1022, 'sea_level': 1022, 'grnd_level': 1003, 'humidity': 65, 'temp_kf': 0.81}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'clouds': {'all': 49}, 'wind': {'speed': 2.16, 'deg': 19, 'gust': 4.61}, 'visibility': 10000, 'pop': 0, 'sys': {'pod'

## An Agent to Read News Headlines

In [93]:
import os
import uuid
import requests
import xml.etree.ElementTree as ET
from dataclasses import dataclass, field
from typing import List, Optional
from agents import function_tool, Agent, Runner

# -------------------------------
# Dataclasses for structured output
# -------------------------------

@dataclass
class NewsHeadline:
    title: str
    link: str
    pub_date: str
    source: str
    summary: str

@dataclass
class NewsFeed:
    topic: str
    headlines: List[NewsHeadline] = field(default_factory=list)

@dataclass
class MessageState:
    session_id: str
    news_feed: Optional[NewsFeed] = None

# -------------------------------
# Helper to safely get XML nodes
# -------------------------------

def get_item_text(element, tag):
    node = element.find(tag)
    return node.text if node is not None else ""

# -------------------------------
# Tool: Fetch Google News Headlines
# -------------------------------

@function_tool
def get_google_news(state: MessageState, topic: str = "world") -> str:
    """
    Fetch top 3 Google News headlines with:
    - title
    - publication date
    - source
    - 100-word summary
    - article URL
    """

    topic_map = {
        "world": "CAASAhgD",
        "business": "CAASBwgD",
        "technology": "CAASBwgC",
        "sports": "CAASBwgE",
        "science": "CAASBwgF",
        "health": "CAASBwgG",
    }

    if topic not in topic_map:
        return f"Unknown topic '{topic}'. Valid topics: {', '.join(topic_map.keys())}"

    rss_url = f"https://news.google.com/rss?hl=en-US&gl=US&ceid=US:{topic_map[topic]}"

    # Fetch the RSS feed
    try:
        resp = requests.get(rss_url, timeout=10)
        resp.raise_for_status()
        xml_data = resp.text
    except Exception as e:
        return f"Error fetching Google News: {str(e)}"

    # Parse XML
    try:
        root = ET.fromstring(xml_data)
    except Exception:
        return "Error parsing the RSS feed."

    headlines = []
    count = 0

    for item in root.iter("item"):
        if count >= 3:
            break

        title = get_item_text(item, "title")
        link = get_item_text(item, "link")
        pub_date = get_item_text(item, "pubDate")

        source_el = item.find("{http://search.yahoo.com/mrss/}source")
        source = source_el.text if source_el is not None else "Google News"

        # Raw description text
        desc_raw = get_item_text(item, "description")
        cleaned = " ".join(desc_raw.split())  # remove newlines, collapse spaces

        # 100-word summary
        words = cleaned.split()
        summary_words = words[:100]  # first 100 words
        summary = " ".join(summary_words)
        if len(words) > 100:
            summary += "..."

        headlines.append(
            NewsHeadline(
                title=title,
                link=link,
                pub_date=pub_date,
                source=source,
                summary=summary
            )
        )

        count += 1

    state.news_feed = NewsFeed(topic=topic, headlines=headlines)

    # -------------------------------
    # Build human-readable output
    # -------------------------------

    output = [f"Top 3 Google News headlines for topic '{topic}':\n"]

    for h in headlines:
        output.append(f"Title: {h.title}")
        output.append(f"Source: {h.source}")
        output.append(f"Date: {h.pub_date}")
        output.append(f"URL: {h.link}")
        output.append(f"Summary (100 words): {h.summary}\n")

    return "\n".join(output)


# -------------------------------
# Agent Setup
# -------------------------------

news_agent = Agent[MessageState](
    name="Google News Agent",
    instructions="""
    You read news headlines using the get_google_news tool.
    ALWAYS call the tool when the user requests news.
    """,
    model="gpt-4o-mini",
    tools=[get_google_news]
)


# -------------------------------
# Run Example
# -------------------------------

async def run_test():
    context = MessageState(session_id=str(uuid.uuid4()))

    response = await Runner.run(
        news_agent,
        [{"role": "user", "content": "Show me the top world news"}],
        context=context
    )

    print("[DEBUG] Agent Final Output:\n")
    print(response.final_output)

    if context.news_feed:
        print("\n[DEBUG] Parsed Headlines:\n")
        for h in context.news_feed.headlines:
            print(h.title, "|", h.source, "|", h.pub_date)


await run_test()


[DEBUG] Agent Final Output:

Here are the top world news headlines:

### 1. [For Marjorie Taylor Greene, a Rough Education in MAGA Politics](https://news.google.com/rss/articles/CBMif0FVX3lxTE9MZGpTMXFyc2Mzc3JjczBFSTZ6WXdEVU9xNWtDX1pfZFNlRktMZmdIWTBJcWFxOEpvVkljSC1jTUhCVTJ3ZWptU1BxMXZ3Wm5Dc3MzbzlYcGFoNVBRaWp2RW9FMHRqd1VKN3F3S0dieGRyN3Fuc3h3VVljMFVPUG8?oc=5)  
   **Source:** The New York Times  
   **Date:** Sat, 22 Nov 2025  
   **Summary:** The article explores the challenges faced by Marjorie Taylor Greene within MAGA politics as she navigates party dynamics, shifting alliances, and her recent resignation from Congress, indicating the impact on her political future and relevance in the GOP.

### 2. [Lawmakers deny US backed latest Ukraine peace plan](https://news.google.com/rss/articles/CBMiiwFBVV95cUxQUUtBeWdEWFRncF92eF96dmlOQm9BZ2pYWk1yUGRvYlBEWk9tOVhqSW1HSURwNUxuWjB1eGNSUVQ2N3BjZDc5d3l6WGN6VWdUMWs2YXpnME9HbjItYWF4YU5MV2RYS2tMdjdPTnhIMHdQOUE0cGF5aUhuYmRzMmUybEtHeTZXUnRMX19B?oc=5)  